<a href="https://colab.research.google.com/github/Aishwaryaa128/aish_portfolio.github.io/blob/main/soup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'}).text.strip()

    except AttributeError:
        title= ""

    return title

# Function to extract Product Price
def get_price(soup):
    try:
        # Try different potential class names that might contain the price
        price = soup.find("span", attrs={'class': 'a-price-whole'}).text.strip()
    except AttributeError:
        try:
            price = soup.find("span", attrs={'class': 'a-offscreen'}).text.strip()
        except AttributeError:
            price = ""
    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [ ]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5','Accept-Encoding': 'gzip, deflate, br',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Referer': 'https://www.amazon.com',
        'DNT': '1',  # Do Not Track Request Header
        'Connection': 'keep-alive'})

    # The webpage URL
    URL="https://www.amazon.in/s?k=phone+under+1000%2B0&crid=3J8T4L98G7Y4Z&sprefix=phone%2Caps%2C227&ref=nb_sb_ss_pltr-data-refreshed_3_5"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [ ]:
amazon_df.head()

,title,price,rating,reviews,availability
0,"Samsung Galaxy M14 4G (Arctic Blue,4GB,64GB) |...","8,499.",3.9 out of 5 stars,157 ratings,Only 1 left in stock.
1,POCO C65 (Pastel Blue 6GB RAM 128GB Storage),"7,499.",4.0 out of 5 stars,"1,105 ratings",In stock
2,"Redmi 13C (Starfrost White, 6GB RAM, 128GB Sto...","8,499.",4.1 out of 5 stars,"5,708 ratings",In stock
3,Nokia All-new 105 Single Sim Keypad Phone with...,"1,199.",4.0 out of 5 stars,"143,688 ratings",In stock
4,"realme NARZO N63 (Leather Blue, 4GB RAM+128GB ...","8,998.",4.2 out of 5 stars,114 ratings,In stock


In [ ]:
amazon_df.shape

(16, 5)